In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Example Data Load

Loads some example data into LUSID

Attributes
----------
transactions
cutlabels
quotes
valuation
portfolio groups

"""

toggle_code("Toggle Docstring")

# Example Data Load

In [2]:
## Importing required packages
import os
import pandas as pd
import datetime
from datetime import timezone
import pytz
import requests.exceptions

import lusid
import lusid.models as models
from lusidjam.refreshing_token import RefreshingToken
import fbnsdkutilities.utilities as utils

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = utils.ApiClientFactory(
    lusid,
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [3]:
# construct the API we want to use
cut_label_definitions_api = api_factory.build(lusid.api.CutLabelDefinitionsApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
portfolio_groups_api = api_factory.build(lusid.api.PortfolioGroupsApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
valuations_api = api_factory.build(lusid.api.AggregationApi)

In [4]:
scope = "Finbourne-Examples"
market_data_scope = "FinbourneMarketData"

datafile = "ExampleData.xlsx"
date = datetime.datetime(2021, 1, 1, 0, 0, 0, 0, tzinfo=timezone.utc)

In [5]:
## Loading Data
portfolios = pd.read_excel(datafile,sheet_name = 'Portfolios')
holdings = pd.read_excel(datafile,sheet_name = 'Holdings')
transactions = pd.read_excel(datafile,sheet_name = 'Transactions')
instruments = pd.read_excel(datafile,sheet_name = 'Instruments')
quotes = pd.read_excel(datafile,sheet_name = 'Quotes').fillna('')
rates = pd.read_excel(datafile,sheet_name = 'Rates').fillna('')
recipes = pd.read_excel(datafile,sheet_name = 'Recipes')
groups = pd.read_excel(datafile,sheet_name = 'Groups')

/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### Creating CutLabels

In [6]:
cut_labels = [["LDN_Open","LondonOpen","London Market Open Time", models.CutLocalTime(8, 0), "Europe/London"],
             ["LDN_Close","LondonClose","London Market Close Time", models.CutLocalTime(16, 30), "Europe/London"],
             ["NY_Close","NewYorkClose","New York Market Close Time", models.CutLocalTime(16, 00), "America/New_York"],
             ["NY_Open","NewYorkOpen","New York Market Open Time", models.CutLocalTime(9, 30), "America/New_York"]]

def create_cut_label(cut_label, overwrite=True):
    '''
    Input: 
        cut_label (list): ordered list where 
                            [0] == code (str), 
                            [1] == display_name (str), 
                            [2] == description (str), 
                            [3] == cut_local_time (models.cutLabelTime),
                            [4] == time_zone (str)
        overwrite (bool): whether to overwrite existing cutlabel in the provided code if it exists, defaults to True
    Output:
        resp (lusid.models.cut_label_definition.CutLabelDefinition): Cut Label definition of the created cut label
    '''

    cut_label_req = models.CreateCutLabelDefinitionRequest(code=cut_label[0],
                                                     display_name=cut_label[1],
                                                     description=cut_label[2],
                                                     cut_local_time=cut_label[3],
                                                     time_zone=cut_label[4])
    
    ## Try creating cutLabel, throws exception if already exists
    try:
        cut_label_definitions_api.create_cut_label_definition(create_cut_label_definition_request=cut_label_req)
    except:
        if overwrite:
            resp = cut_label_definitions_api.delete_cut_label_definition(code=cut_label[0])
            cut_label_definitions_api.create_cut_label_definition(create_cut_label_definition_request=cut_label_req)
            
    return cut_label_definitions_api.get_cut_label_definition(code=cut_label[0])

created_cut_labels = [create_cut_label(c) 
                      for c in cut_labels]
        
created_cut_labels

[{'code': 'LDN_Open',
  'cut_local_time': {'hours': 8, 'minutes': 0},
  'description': 'London Market Open Time',
  'display_name': 'LondonOpen',
  'href': None,
  'links': [{'description': None,
             'href': 'https://fbn-ci.lusid.com/api/api/schemas/entities/CutLabelDefinition',
             'method': 'GET',
             'relation': 'EntitySchema'},
            {'description': 'A link to the LUSID Insights website showing all '
                            'logs related to this request',
             'href': 'http://fbn-ci.lusid.com/app/insights/logs/0HMQGGAL4G36Q:0000007F',
             'method': 'GET',
             'relation': 'RequestLogs'}],
  'time_zone': 'Europe/London'},
 {'code': 'LDN_Close',
  'cut_local_time': {'hours': 16, 'minutes': 30},
  'description': 'London Market Close Time',
  'display_name': 'LondonClose',
  'href': None,
  'links': [{'description': None,
             'href': 'https://fbn-ci.lusid.com/api/api/schemas/entities/CutLabelDefinition',
           

### Creating Instruments

In [7]:
def create_instrument(instrument, overwrite=True):
    '''
    Input: 
        instrument (dict): dictionary with fields 
                            'name' == name of the instrument (str), 
                            'Figi' == Figi identifier (str), 
                            'P:Instrument/default/Currency' == default currency (str)
        overwrite (bool): whether to overwrite existing instrument (hard-delete) in the provided code if it exists, defaults to True
    Output:
        resp (lusid.models.instrument.Instrument): Instrument that was created
    '''
    ## Checks if Instrument already exist
    try:
        # Check if Instrument with FIGI already exist
        resp = instruments_api.get_instrument("Figi",instrument['Figi'])
        if overwrite:
            resp = instruments_api.delete_instruments(request_body = [resp.lusid_instrument_id],delete_mode = 'Hard')
        else:
            return resp

    except:
        pass

    instrument_req = {"inst":models.InstrumentDefinition(
                                    name=instrument["name"],
                                    identifiers={"Figi":models.InstrumentIdValue(value = instrument['Figi'])},
                                    properties=[lusid.ModelProperty(key="Instrument/default/Currency",
                                                                    value=lusid.PropertyValue(label_value=instrument['P:Instrument/default/Currency'])
                                                                   )
                                               ]
                                    )
                    }
    instruments_api.upsert_instruments(request_body=instrument_req)
    return instruments_api.get_instrument("Figi",instrument['Figi'],property_keys=['Instrument/default/Currency'])

created_instruments = [create_instrument(inst.to_dict()) 
                       for i,inst in instruments.iterrows()]
    
created_instruments

[{'asset_class': 'Unknown',
  'dom_ccy': 'ZZZ',
  'href': 'https://fbn-ci.lusid.com/api/api/instruments/LusidInstrumentId/LUID_00070D4R?scope=default',
  'identifiers': {'Figi': 'BBG000B9XRY4', 'LusidInstrumentId': 'LUID_00070D4R'},
  'instrument_definition': None,
  'links': [{'description': None,
             'href': 'https://fbn-ci.lusid.com/api/api/schemas/entities/Instrument',
             'method': 'GET',
             'relation': 'EntitySchema'},
            {'description': None,
             'href': 'https://fbn-ci.lusid.com/api/api/schemas/properties?propertyKeys=Instrument%2Fdefault%2FCurrency',
             'method': 'GET',
             'relation': 'PropertySchema'},
            {'description': 'A link to the LUSID Insights website showing all '
                            'logs related to this request',
             'href': 'http://fbn-ci.lusid.com/app/insights/logs/0HMQGGAL4G30V:00000156',
             'method': 'GET',
             'relation': 'RequestLogs'}],
  'lookthroug

### Creating Portfolios

In [8]:
def create_portfolio(portfolio, overwrite=True):
    '''
    Input: 
        portfolio (dict): dictionary with fields 
                            'Code' == Code for the portfolio (str), 
                            'Display Name' == Display name of the identifier (str), 
                            'Base Currency' == Base Currency (str)
        overwrite (bool): whether to overwrite existing portfolio (hard-delete) in the provided code if it exists, defaults to True
    Output:
        resp (lusid.models.portfolio.Portfolio): Portfolio that was created
    '''
    port_req = models.CreateTransactionPortfolioRequest(code=portfolio['Code'],
                                                       display_name=portfolio['Display Name'],
                                                       base_currency=portfolio['Base Currency'],
                                                       created=date)
    
    ## Try creating portfolio, throws exception if already exists
    try:
        transaction_portfolios_api.create_portfolio(scope=scope,create_transaction_portfolio_request=port_req)
    except:
        if overwrite:
            portfolios_api.delete_portfolio(scope=scope,code=portfolio['Code'])
            transaction_portfolios_api.create_portfolio(scope=scope,create_transaction_portfolio_request=port_req)
    
    return portfolios_api.get_portfolio(scope=scope,code=portfolio['Code'])

created_portfolios = [create_portfolio(port.to_dict()) 
                      for i,port in portfolios.iterrows()]

created_portfolios

[{'accounting_method': 'Default',
  'amortisation_method': 'NoAmortisation',
  'base_currency': 'GBP',
  'created': datetime.datetime(2021, 1, 1, 0, 0, tzinfo=tzlocal()),
  'description': None,
  'display_name': 'UK Equity Fund',
  'href': 'https://fbn-ci.lusid.com/api/api/portfolios/Finbourne-Examples/UK-Equities?effectiveAt=2023-05-09T12%3A54%3A15.7244980%2B00%3A00&asAt=2023-05-09T12%3A54%3A15.3480880%2B00%3A00',
  'id': {'code': 'UK-Equities', 'scope': 'Finbourne-Examples'},
  'instrument_scopes': [],
  'is_derived': False,
  'links': [{'description': None,
             'href': 'https://fbn-ci.lusid.com/api/api/portfolios/Finbourne-Examples/UK-Equities/properties?effectiveAt=2023-05-09T12%3A54%3A15.7244980%2B00%3A00&asAt=2023-05-09T12%3A54%3A15.3480880%2B00%3A00',
             'method': 'GET',
             'relation': 'Properties'},
            {'description': None,
             'href': 'https://fbn-ci.lusid.com/api/api/portfolios/Finbourne-Examples/UK-Equities/commands?toAsAt=2023-

### Creating Portfolio Groups

In [9]:
port_groups = groups[['Group Name', 'Display Name', 'Description','Created Date']].drop_duplicates().reset_index(drop=True)
port_groups['Sub-groups'] = [list(set(groups.query(f"`Group Name`=='{grp['Group Name']}' and `Display Name`=='{grp['Display Name']}' and Description == '{grp['Description']}' and `Created Date` == '{grp['Created Date']}' ").dropna(subset=['Sub-groups'])['Sub-groups'])) 
                            for i,grp in port_groups.iterrows()]
port_groups['Portfolios'] = [list(set(groups.query(f"`Group Name`=='{grp['Group Name']}' and `Display Name`=='{grp['Display Name']}' and Description == '{grp['Description']}' and `Created Date` == '{grp['Created Date']}' ").dropna(subset=['Portfolios'])['Portfolios'])) 
                            for i,grp in port_groups.iterrows()]
portfolio_groups = [port_groups.iloc[i].to_dict() for i in range(len(port_groups))]
portfolio_groups.reverse()

def create_portfolio_group(portfolio_group, overwrite=True):
    '''
    Input: 
        portfolio_group (dict): dictionary with fields 
                                'Group Name': code of the portfolio group (str),
                                'Display Name': Display name of the portfolio group (str),
                                'Description': Description of the portfolio group (str),
                                'Created Date': Date of creation in YYYY-MM-DDTHH:mm:ssZ (str),
                                'Sub-groups': list of codes of sub portfolio groups that must exist (list[(str)]),
                                'Portfolios': list of codes of portfolio that must exist (list[(str)])
        overwrite (bool): whether to overwrite existing portfolioGroup (hard-delete) in the provided code if it exists, defaults to True
    Output:
        resp (lusid.models.portfolio_group.PortfolioGroup): portfolio_group that was created
    '''
    port_grp_req = models.CreatePortfolioGroupRequest(code=portfolio_group['Group Name'],
                                                    created=portfolio_group['Created Date'],
                                                    values=[lusid.ResourceId(scope=scope, code = port) 
                                                            for port in portfolio_group['Portfolios']], 
                                                    sub_groups=[lusid.ResourceId(scope=scope, code = subGrp) 
                                                                for subGrp in portfolio_group['Sub-groups']],
                                                    description=portfolio_group['Description'],
                                                    display_name=portfolio_group['Display Name'])
    
    ## Try creating portfolio, throws exception if already exists
    try:
        portfolio_groups_api.create_portfolio_group(scope=scope,create_portfolio_group_request=port_grp_req)
    except:
        if overwrite:
            portfolio_groups_api.delete_portfolio_group(scope=scope, code= portfolio_group['Group Name'])
            portfolio_groups_api.create_portfolio_group(scope=scope,create_portfolio_group_request=port_grp_req)
    
    return portfolio_groups_api.get_portfolio_group(scope=scope, code= portfolio_group['Group Name'])

created_portfolio_groups = [create_portfolio_group(grp) 
                            for grp in portfolio_groups]
    
created_portfolio_groups

[{'created': datetime.datetime(2021, 1, 1, 0, 0, tzinfo=tzlocal()),
  'description': 'Contains Finbourne US Fixed Income portfolios',
  'display_name': 'US Fixed Income group',
  'href': 'https://fbn-ci.lusid.com/api/api/portfoliogroups/Finbourne-Examples/US-FI?effectiveAt=2021-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-05-09T12%3A54%3A20.7615180%2B00%3A00',
  'id': {'code': 'US-FI', 'scope': 'Finbourne-Examples'},
  'links': [{'description': None,
             'href': 'https://fbn-ci.lusid.com/api/api/schemas/entities/PortfolioGroup',
             'method': 'GET',
             'relation': 'EntitySchema'},
            {'description': 'A link to the LUSID Insights website showing all '
                            'logs related to this request',
             'href': 'http://fbn-ci.lusid.com/app/insights/logs/0HMQGGAL4G36Q:000000CD',
             'method': 'GET',
             'relation': 'RequestLogs'}],
  'portfolios': [{'code': 'US-Treasury-Bond', 'scope': 'Finbourne-Examples'},
   

### Set Holdings

In [10]:
def set_holdings(scope, port, port_holdings):
    '''
    Input: 
        port_holdings (pandas.DataFrame): pandas dataframe with the following columns 
                                'Portfolio Code': code of the portfolio (str),
                                'instrument_uid': identifier of the instrument (identifierkey:value) (dict),
                                'Instrument Name': name of the instrument (str),
                                'Holding Type': type of the holding (str),
                                'units': number of the given instrument (float),
                                'price': price of the instrument (float),
                                'cost.currency': CCY code (str),
                                'purchase_date': purchase date in UTC (pandas._libs.tslibs.timestamps.Timestamp),
                                'settlement_date': settlmeent date in UTC (pandas._libs.tslibs.timestamps.Timestamp),
                                'cost.amount': cost of the transaction to get this holding in cost currency (float),
                                'portfolio_cost': cost of the transaction to get this holding in portfolio currency (float)
    Output:
        resp (lusid.models.adjust_holding.AdjustHolding): AdjustHolding response
    '''
    holdingsReq = [models.AdjustHoldingRequest(instrument_identifiers={hold['instrument_uid'].split(':')[0]:hold['instrument_uid'].split(':')[1]},
                                           tax_lots=[models.TargetTaxLotRequest(units=hold['units'],
                                                                                cost=models.CurrencyAndAmount(amount=hold['cost.amount'],
                                                                                                              currency=hold['cost.currency']),
                                                                                portfolio_cost=hold['portfolio_cost'],
                                                                                price=hold['price'],
                                                                                purchase_date=pytz.timezone('UTC').localize(hold['purchase_date'].to_pydatetime()).isoformat(),
                                                                                settlement_date=pytz.timezone('UTC').localize(hold['settlement_date'].to_pydatetime()).isoformat())],
                                          currency=hold['cost.currency']) 
               for i,hold in port_holdings.iterrows()]

    resp = transaction_portfolios_api.set_holdings(scope=scope,
                                          code=port,
                                          effective_at=date.isoformat(),
                                          adjust_holding_request=holdingsReq)
    return resp

set_holdings_response = [set_holdings(scope, port, holdings.query(f"`Portfolio Code` == '{port}'")) 
                         for port in holdings['Portfolio Code'].unique()]
    
set_holdings_response

[{'href': 'https://fbn-ci.lusid.com/api/api/transactionportfolios/Finbourne-Examples/Global-Equity/holdings?asAt=2023-05-09T12%3A54%3A16.7134140%2B00%3A00',
  'links': [{'description': None,
             'href': 'https://fbn-ci.lusid.com/api/api/portfolios/Finbourne-Examples/Global-Equity?effectiveAt=2021-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-05-09T12%3A54%3A16.7134140%2B00%3A00',
             'method': 'GET',
             'relation': 'Root'},
            {'description': None,
             'href': 'https://fbn-ci.lusid.com/api/api/transactionportfolios/Finbourne-Examples/Global-Equity/holdings?effectiveAt=2021-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-05-09T12%3A54%3A16.7134140%2B00%3A00',
             'method': 'GET',
             'relation': 'Holdings'},
            {'description': 'A link to the LUSID Insights website showing all '
                            'logs related to this request',
             'href': 'http://fbn-ci.lusid.com/app/insights/logs/0HMQGGAL4G34J:0

### Create Transactions

In [11]:
upsert_transaction_response = []

def upsert_transactions(scope, port, port_trans):
    '''
    Input: 
        port_trans (pandas.DataFrame): pandas dataframe with the following columns 
                                'Portfolio Code': code of the portfolio (str),
                                'transaction_id' : ID of this transaction (str),
                                'instrument_uid': identifier of the instrument (identifierkey:value) (dict),
                                'type' : type of this transaction (str),
                                'transaction_date' : transaction date in UTC (pandas._libs.tslibs.timestamps.Timestamp),
                                'settlement_date': settlement date in UTC (pandas._libs.tslibs.timestamps.Timestamp),
                                'units': number of the given instrument (float),
                                'total_consideration.currency' : CCY code (str),
                                'transaction_price.price' : price at which the instrument was transacted at (float),
                                'total_consideration.amount' : amount of the particular instrument transacted (float),
                                'source : source of this transaction (str)'
    Output:
        resp (lusid.models.adjust_holding.AdjustHolding): AdjustHolding response
    '''
    trans_req = [models.TransactionRequest(transaction_id=trans['transaction_id'], 
                                          type=trans['type'],
                                          instrument_identifiers={trans['instrument_uid'].split(':')[0]:trans['instrument_uid'].split(':')[1]}, 
                                          transaction_date=pytz.timezone('UTC').localize(trans['transaction_date'].to_pydatetime()).isoformat(),
                                          settlement_date=pytz.timezone('UTC').localize(trans['settlement_date'].to_pydatetime()).isoformat(), 
                                          units=trans['units'],
                                          transaction_price=models.TransactionPrice(price=trans['transaction_price.price']),
                                          total_consideration=models.CurrencyAndAmount(amount=trans['total_consideration.amount'],
                                                                                       currency=trans['total_consideration.currency']),
                                          source=trans['source'])
                for i,trans in port_trans.iterrows()]

    resp = transaction_portfolios_api.upsert_transactions(scope=scope,
                                                        code=port,
                                                        transaction_request=trans_req)
    return resp


upsert_transaction_response = [upsert_transactions(scope, port, transactions.query(f"`Portfolio Code` == '{port}'")) 
                               for port in transactions['Portfolio Code'].unique()]    

upsert_transaction_response

[{'href': 'https://fbn-ci.lusid.com/api/api/transactionportfolios/Finbourne-Examples/Global-Equity/transactions?asAt=2023-05-09T12%3A54%3A26.1059000%2B00%3A00',
  'links': [{'description': None,
             'href': 'https://fbn-ci.lusid.com/api/api/portfolios/Finbourne-Examples/Global-Equity?effectiveAt=2021-05-02T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-05-09T12%3A54%3A26.1059000%2B00%3A00',
             'method': 'GET',
             'relation': 'Root'},
            {'description': None,
             'href': 'https://fbn-ci.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
             'method': 'GET',
             'relation': 'EntitySchema'},
            {'description': 'A link to the LUSID Insights website showing all '
                            'logs related to this request',
             'href': 'http://fbn-ci.lusid.com/app/insights/logs/0HMQGGAL4G383:00000020',
             'method': 'GET',
             'relation': 'RequestLogs'}],
  'metadata': {},
  'v

### Create Quotes

In [12]:
quotes_req = {
    f"{quote['instrument_uid']}-{str(quote['effective_at'])}-{quote['field']}":
    lusid.UpsertQuoteRequest(
        quote_id=lusid.QuoteId(
            quote_series_id = lusid.QuoteSeriesId(provider = quote['provider'],
                                                  price_source=quote["source"],
                                                  instrument_id = quote['instrument_uid'], 
                                                  instrument_id_type=quote['instrument_uid_type'], 
                                                  quote_type=quote['quote_type'], 
                                                  field=quote['field']),
            effective_at= pytz.timezone('UTC').localize(quote['effective_at'].to_pydatetime()).isoformat()), 
        metric_value=lusid.MetricValue(value=quote['metric_value'],
                                       unit=quote['metric_unit']))
    for i,quote in quotes.iterrows()
}
    
## Upserting Quotes
quote_numbers = len(quotes_req)
max_per_upsert = 500
number_of_Req = quote_numbers/max_per_upsert
quote_keys = list(quotes_req.keys())
upsert_results = []
for i in range(int(number_of_Req)):
    request_keys = quote_keys[i*max_per_upsert : (i+1)*max_per_upsert]
    request = dict(zip(request_keys, [quotes_req[key] for key in request_keys]))
    upsert_results.append(quotes_api.upsert_quotes(scope=market_data_scope, request_body=request))
    
if quote_numbers - ((i+1)*max_per_upsert) > 0:
    request_keys = quote_keys[((i+1)*max_per_upsert) : ]
    request = dict(zip(request_keys, [quotes_req[key] for key in request_keys]))
    upsert_results.append(quotes_api.upsert_quotes(scope=market_data_scope, request_body=request))

    
print("Failures: ", [i.failed for i in upsert_results])

Failures:  [{}, {}, {}, {}, {}]


### Create FX Rates

In [13]:
rates_req = {
    f"{rate['instrument_uid']}-{str(rate['effective_at'])}-{rate['field']}" :
    lusid.UpsertQuoteRequest(
        quote_id=lusid.QuoteId(
            quote_series_id = lusid.QuoteSeriesId(provider = 'Client', 
                                                  price_source=rate["source"],
                                                  instrument_id = rate['instrument_uid'], 
                                                  instrument_id_type=rate['instrument_uid_type'], 
                                                  quote_type=rate['quote_type'], 
                                                  field=rate['field']),
            effective_at = pytz.timezone('UTC').localize(rate['effective_at'].to_pydatetime()).isoformat()), 
        metric_value=lusid.MetricValue(value=rate['metric_value'],
                                       unit=rate['metric_unit']))
    for i,rate in rates.iterrows()
}

## Upserting Quotes
quote_numbers = len(rates_req)
max_per_upsert = 500
number_of_Req = quote_numbers/max_per_upsert
quote_keys = list(rates_req.keys())
upsert_results = []
for i in range(int(number_of_Req)):
    request_keys = quote_keys[i*max_per_upsert : (i+1)*max_per_upsert]
    request = dict(zip(request_keys, [rates_req[key] for key in request_keys]))
    upsert_results.append(quotes_api.upsert_quotes(scope=market_data_scope, request_body=request))
    
if quote_numbers - ((i+1)*max_per_upsert) > 0:
    request_keys = quote_keys[((i+1)*max_per_upsert) : ]
    request = dict(zip(request_keys, [rates_req[key] for key in request_keys]))
    upsert_results.append(quotes_api.upsert_quotes(scope=market_data_scope, request_body=request))

    
print("Failures: ", [i.failed for i in upsert_results])

Failures:  [{}, {}, {}, {}, {}]


### Create Recipes

In [14]:
created_recipes = []

def create_recipe(code, rec_rules):
    '''
    Input: 
        rec_rules (pandas.DataFrame): pandas dataframe with the following columns 
                                'Description' : description of the recipe (str),
                                'Key' : market data key (str),
                                'Supplier' : supplier of the market data (str),
                                'QuoteType' : type of the quote (str),
                                'QuoteInterval' : interval of the quote (str),
                                'Field' : field of the quote (str)
    Output:
        resp (lusid.models.adjust_holding.AdjustHolding): AdjustHolding response
    '''
    description = rec_rules.Description.unique()[0]
    supplier = "Client" #rec_rules.Supplier.unique()[0]
    recipe_req = models.UpsertRecipeRequest(
                        configuration_recipe=lusid.ConfigurationRecipe(
                            scope=scope,
                            code=code,
                            description=description,
                            market=models.MarketContext(market_rules=[
                                models.MarketDataKeyRule(key=rule['Key'],
                                   supplier=rule['Supplier'],
                                   data_scope=market_data_scope,
                                   quote_type=rule['QuoteType'],
                                   field=rule['Field'],
                                   quote_interval=rule['QuoteInterval'])
                            for i,rule in rec_rules.iterrows()],
                            suppliers=lusid.MarketContextSuppliers(
                                commodity=supplier,
                                credit=supplier,
                                equity=supplier,
                                fx=supplier,
                                rates=supplier),
                           options=models.MarketOptions(
                                default_supplier='DataScope',
                                default_instrument_code_type='Figi',
                                default_scope=market_data_scope)
                               )
                        )
                ) 

    configuration_recipe_api.upsert_configuration_recipe(upsert_recipe_request = recipe_req)
    return configuration_recipe_api.get_configuration_recipe(scope=scope, code=code)

created_recipes = [create_recipe(rec, recipes.query(f"Code == '{rec}'")) 
                   for rec in recipes['Code'].unique()]

created_recipes

[{'href': None,
  'links': [{'description': 'A link to the LUSID Insights website showing all '
                            'logs related to this request',
             'href': 'http://fbn-ci.lusid.com/app/insights/logs/0HMQGGAL4G383:0000003D',
             'method': 'GET',
             'relation': 'RequestLogs'}],
  'value': {'aggregation': {'options': {'allow_partial_entitlement_success': False,
                                        'apply_iso4217_rounding': False,
                                        'use_ansi_like_syntax': False}},
            'code': 'FinbourneExamplesRecipeMidThenBid',
            'description': 'For each instrument (Equity and Fx) use Mid Price '
                           'if available, Bid if not',
            'holding': {'tax_lot_level_holdings': True},
            'inherited_recipes': [],
            'market': {'market_rules': [{'as_at': None,
                                         'data_scope': 'FinbourneMarketData',
                                 

In [15]:
[i.value.code for i in created_recipes]

['FinbourneExamplesRecipeMidThenBid',
 'FinbourneExamplesRecipeBidThenMid',
 'FinbourneExamplesRecipeLusidFlatFX']

### Test Aggregation using created Recipe

In [16]:
AGG_INSTR = "Instrument/default/Name"
AGG_UID = "Instrument/default/LusidInstrumentId"
AGG_PV = "Holding/default/PV"
AGG_PRC = "Holding/default/Price"
AGG_UNITS = "Holding/default/Units"
AGG_COST = "Holding/default/Cost"
AGG_TYPE = "Holding/default/Type"
AGG_RATE = "Holding/default/ExchangeRate"


metrics = [
    models.AggregateSpec(AGG_INSTR, "Value"),
    models.AggregateSpec(AGG_UID, "Value"),
    models.AggregateSpec(AGG_TYPE, "Value"),
    models.AggregateSpec(AGG_PV, "Value"),
    models.AggregateSpec(AGG_UNITS, "Value"),
    models.AggregateSpec(AGG_COST, "Value"),
    models.AggregateSpec(AGG_RATE, "Value"),
    models.AggregateSpec(AGG_PRC, "Value"),
    ]


valuation_request = models.ValuationRequest(
    recipe_id=models.ResourceId(scope, created_recipes[0].value.code),
    valuation_schedule=models.ValuationSchedule(effective_at=pd.to_datetime('2021-05-17', utc=True).isoformat()),
    portfolio_entity_ids=[
        models.PortfolioEntityId(
                scope=created_portfolios[1].id.scope,
                code=created_portfolios[1].id.code,
                portfolio_entity_type="SinglePortfolio")
    ],
    metrics=metrics
)

valuations_api.get_valuation(valuation_request=valuation_request)

{'aggregation_as_at': datetime.datetime(2023, 5, 9, 12, 54, 58, 892719, tzinfo=tzlocal()),
 'aggregation_currency': 'GBP',
 'aggregation_effective_at': datetime.datetime(2021, 5, 17, 0, 0, tzinfo=tzlocal()),
 'aggregation_failures': [],
 'data': [{'Holding/default/Cost': 2390760000.0,
           'Holding/default/ExchangeRate': 1.0,
           'Holding/default/PV': 2268000000.0,
           'Holding/default/Price': 189.0,
           'Holding/default/Type': 'Position',
           'Holding/default/Units': 12000000.0,
           'Instrument/default/LusidInstrumentId': 'LUID_00070D4R',
           'Instrument/default/Name': 'Apple'},
          {'Holding/default/Cost': 2379056470.59,
           'Holding/default/ExchangeRate': 1.0,
           'Holding/default/PV': 2365265294.117647,
           'Holding/default/Price': 110.77,
           'Holding/default/Type': 'Position',
           'Holding/default/Units': 21352941.17647059,
           'Instrument/default/LusidInstrumentId': 'LUID_00070D4S',
 